In [1]:
%load_ext autoreload
%autoreload 2

We need first install tensorflow_model_optimization, then needed tensorflow version, then tf_keras,  then needed tensorflow-io version   

tensorflow-addons ?

In [2]:
import tensorflow as tf

2024-07-28 17:49:56.305475: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-28 17:49:56.425319: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-28 17:49:56.561000: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-28 17:49:56.734805: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-28 17:49:56.795183: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-28 17:49:56.996012: I tensorflow/core/platform/cpu_feature_gu

In [3]:
# import tensorflow_io
# from tensorflow_model_optimization.python.core.keras.compat import keras
# from tensorflow_model_optimization.quantization import keras


In [4]:
import tensorflow as tf
# from tensorflow_model_optimization.python.core.keras.compat import keras
import tensorflow_model_optimization as tfmot
import numpy as np
import matplotlib.pyplot as plt

import sys 
sys.path.append("C:/Users/polin/TinyBirdSounds") 

from create_spectrogram import (
    create_spectrograms_from_audio_dataset, 
)
from helper_functions import (
    get_file_size, 
    convert_bytes, 
    convert_prefetchdataset_to_numpy_arrays,
    predict_and_print_full_results,
    evaluate_time_of_prediction
    )

In [5]:
train_dataset = tf.keras.utils.audio_dataset_from_directory("../dataset/training", labels='inferred', sampling_rate=16000, batch_size=32, shuffle=True, seed=3407)
test_dataset = tf.keras.utils.audio_dataset_from_directory("../dataset/testing", labels='inferred', sampling_rate=16000, batch_size=32, shuffle=True, seed=3407)
val_dataset = tf.keras.utils.audio_dataset_from_directory("../dataset/validation", labels='inferred', sampling_rate=16000, batch_size=32, shuffle=True, seed=3407)

label_names = np.array(train_dataset.class_names)
print("Classes: ", label_names)

Found 11292 files belonging to 2 classes.


I0000 00:00:1722181804.352163   28780 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-28 17:50:04.352721: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2343] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


2024-07-28 17:50:04.557107: I tensorflow_io/core/kernels/cpu_check.cc:128] Your CPU supports instructions that this TensorFlow IO binary was not compiled to use: SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA
2024-07-28 17:50:04.631909: W tensorflow_io/core/kernels/audio_video_mp3_kernels.cc:271] libmp3lame.so.0 or lame functions are not available


Found 1393 files belonging to 2 classes.
Found 1380 files belonging to 2 classes.
Classes:  ['non_target' 'target']


In [6]:
x_train_np, y_train_np = convert_prefetchdataset_to_numpy_arrays(train_dataset, data_type="time-series")
x_val_np, y_val_np = convert_prefetchdataset_to_numpy_arrays(val_dataset, data_type="time-series")
x_test_np, y_test_np = convert_prefetchdataset_to_numpy_arrays(test_dataset, data_type="time-series")

2024-07-28 17:50:28.992701: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-07-28 17:50:34.031772: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [44]:
BASE_MODEL_FILE_NAME = "../time_series_models_from_notebooks/cnn/hpo/cnn_time_ser_3_conv_layer_model.keras"
base_model = tf.keras.models.load_model(BASE_MODEL_FILE_NAME)
base_model.summary()
convert_bytes(get_file_size(BASE_MODEL_FILE_NAME), "MB")

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_21 (Conv1D)          (None, 47996, 4)          24        
                                                                 
 max_pooling1d_21 (MaxPooli  (None, 5999, 4)           0         
 ng1D)                                                           
                                                                 
 conv1d_22 (Conv1D)          (None, 5995, 4)           84        
                                                                 
 max_pooling1d_22 (MaxPooli  (None, 749, 4)            0         
 ng1D)                                                           
                                                                 
 conv1d_23 (Conv1D)          (None, 745, 8)            168       
                                                                 
 max_pooling1d_23 (MaxPooli  (None, 93, 8)           

In [40]:
q_aware_model = tfmot.quantization.keras.quantize_annotate_model(base_model)
q_aware_model = tfmot.quantization.keras.quantize_model(q_aware_model)


RuntimeError: Layer conv1d_21:<class 'tf_keras.src.layers.convolutional.conv1d.Conv1D'> is not supported. You can quantize this layer by passing a `tfmot.quantization.keras.QuantizeConfig` instance to the `quantize_annotate_layer` API.

In [46]:
def apply_quantization_to_conv1d(layer):
    if isinstance(layer, tf.keras.layers.Conv1D):
        return tfmot.quantization.keras.quantize_annotate_layer(layer)
    return layer

annotated_model = tf.keras.models.clone_model(
    base_model,
    clone_function=apply_quantization_to_conv1d
)

# Apply quantization
quant_aware_model = tfmot.quantization.keras.quantize_apply(annotated_model)

RuntimeError: Layer conv1d_21:<class 'tf_keras.src.layers.convolutional.conv1d.Conv1D'> is not supported. You can quantize this layer by passing a `tfmot.quantization.keras.QuantizeConfig` instance to the `quantize_annotate_layer` API.

In [50]:
LastValueQuantizer = tfmot.quantization.keras.quantizers.LastValueQuantizer
MovingAverageQuantizer = tfmot.quantization.keras.quantizers.MovingAverageQuantizer

class DefaultDenseQuantizeConfig(tfmot.quantization.keras.QuantizeConfig):
    # Configure how to quantize weights.
    def get_weights_and_quantizers(self, layer):
      return [(layer.kernel, LastValueQuantizer(num_bits=8, symmetric=True, narrow_range=False, per_axis=False))]

    # Configure how to quantize activations.
    def get_activations_and_quantizers(self, layer):
      return [(layer.activation, MovingAverageQuantizer(num_bits=8, symmetric=False, narrow_range=False, per_axis=False))]

    def set_quantize_weights(self, layer, quantize_weights):
      # Add this line for each item returned in `get_weights_and_quantizers`
      # , in the same order
      layer.kernel = quantize_weights[0]

    def set_quantize_activations(self, layer, quantize_activations):
      # Add this line for each item returned in `get_activations_and_quantizers`
      # , in the same order.
      layer.activation = quantize_activations[0]

    # Configure how to quantize outputs (may be equivalent to activations).
    def get_output_quantizers(self, layer):
      return []

    def get_config(self):
      return {}
    


quantize_annotate_layer = tfmot.quantization.keras.quantize_annotate_layer
quantize_annotate_model = tfmot.quantization.keras.quantize_annotate_model
quantize_scope = tfmot.quantization.keras.quantize_scope

class CustomLayer(tf.keras.layers.Conv1D):
  pass

model = quantize_annotate_model(base_model)

# `quantize_apply` requires mentioning `DefaultDenseQuantizeConfig` with `quantize_scope`
# as well as the custom Keras layer.
with quantize_scope(
  {'DefaultDenseQuantizeConfig': DefaultDenseQuantizeConfig,
   'CustomLayer': CustomLayer}):
  # Use `quantize_apply` to actually make the model quantization aware.
  quant_aware_model = tfmot.quantization.keras.quantize_apply(model)

quant_aware_model.summary()

RuntimeError: Layer conv1d_21:<class 'tf_keras.src.layers.convolutional.conv1d.Conv1D'> is not supported. You can quantize this layer by passing a `tfmot.quantization.keras.QuantizeConfig` instance to the `quantize_annotate_layer` API.

In [49]:
# Define a custom quantization scheme
class CustomQuantizeConfig(tfmot.quantization.keras.QuantizeConfig):
    def get_weights_and_quantizers(self, layer):
      return [(layer.kernel, tfmot.quantization.keras.quantizers.LastValueQuantizer(num_bits=8, symmetric=True, narrow_range=False, per_axis=False))]

    # Configure how to quantize activations.
    def get_activations_and_quantizers(self, layer):
      return [(layer.activation, tfmot.quantization.keras.quantizers.MovingAverageQuantizer(num_bits=8, symmetric=False, narrow_range=False, per_axis=False))]

    def set_quantize_weights(self, layer, quantize_weights):
      # Add this line for each item returned in `get_weights_and_quantizers`
      # , in the same order
      layer.kernel = quantize_weights[0]

    def set_quantize_activations(self, layer, quantize_activations):
      # Add this line for each item returned in `get_activations_and_quantizers`
      # , in the same order.
      layer.activation = quantize_activations[0]

    # Configure how to quantize outputs (may be equivalent to activations).
    def get_output_quantizers(self, layer):
      return []

    def get_config(self):
      return {}

# Apply custom quantization
def apply_custom_quantization_to_layer(layer):
    if isinstance(layer, tf.keras.layers.Conv1D):
        return tfmot.quantization.keras.quantize_annotate_layer(layer, quantize_config=CustomQuantizeConfig())
    return layer

annotated_model = tf.keras.models.clone_model(
    base_model,
    clone_function=apply_custom_quantization_to_layer
)

quant_aware_model = tfmot.quantization.keras.quantize_apply(annotated_model)


ValueError: Unable to clone model. This generally happens if you used custom Keras layers or objects in your model. Please specify them via `quantize_scope` for your calls to `quantize_model` and `quantize_apply`. [Unknown None: 'CustomQuantizeConfig'. Please ensure you are using a `keras.utils.custom_object_scope` and that this object is included in the scope. See https://www.tensorflow.org/guide/keras/save_and_serialize#registering_the_custom_object for details.].

In [27]:
q_aware_model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

q_aware_model.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   


 quantize_annotate_69 (Quan  (None, 47996, 4)          24        
 tizeAnnotate)                                                   
                                                                 
 quantize_annotate_70 (Quan  (None, 5999, 4)           0         
 tizeAnnotate)                                                   
                                                                 
 quantize_annotate_71 (Quan  (None, 5995, 4)           84        
 tizeAnnotate)                                                   
                                                                 
 quantize_annotate_72 (Quan  (None, 749, 4)            0         
 tizeAnnotate)                                                   
                                                                 
 quantize_annotate_73 (Quan  (None, 745, 8)            168       
 tizeAnnotate)                                                   
                                                                 
 quantize_

In [17]:
# quantize_model = tfmot.quantization.keras.quantize_model

# # q_aware stands for for quantization aware.
# q_aware_model = quantize_model(base_model)

# # `quantize_model` requires a recompile.
# q_aware_model.compile(optimizer='adam',
#               loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
#               metrics=['accuracy'])

# q_aware_model.summary()

In [28]:
q_aware_model.fit(x_train_np[:500], y_train_np[:500],
                  batch_size=32, epochs=1, validation_split=0.1)

/home/polina/newname/env_for_qat/lib/python3.9/site-packages/tf_keras/src/backend.py:5729: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


15/15 [==============================] - 2s 82ms/step - loss: 0.0459 - accuracy: 0.9867 - val_loss: 0.0487 - val_accuracy: 0.9800


In [29]:
_, baseline_model_accuracy = base_model.evaluate(
    x_test_np, y_test_np, verbose=0)

_, q_aware_model_accuracy = q_aware_model.evaluate(
   x_test_np, y_test_np, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy)
print('Quant test accuracy:', q_aware_model_accuracy)

Baseline test accuracy: 0.9834888577461243
Quant test accuracy: 0.9834888577461243


In [30]:
converter = tf.lite.TFLiteConverter.from_keras_model(q_aware_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

quantized_tflite_model = converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmp5o1_6vsk/assets


INFO:tensorflow:Assets written to: /tmp/tmp5o1_6vsk/assets
W0000 00:00:1722183734.183097   28780 tf_tfl_flatbuffer_helpers.cc:392] Ignored output_format.
W0000 00:00:1722183734.183156   28780 tf_tfl_flatbuffer_helpers.cc:395] Ignored drop_control_dependency.
2024-07-28 18:22:14.183390: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmp5o1_6vsk
2024-07-28 18:22:14.186437: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2024-07-28 18:22:14.186478: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /tmp/tmp5o1_6vsk
2024-07-28 18:22:14.198835: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2024-07-28 18:22:14.248580: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: /tmp/tmp5o1_6vsk
2024-07-28 18:22:14.273469: I tensorflow/cc/saved_model/loader.cc:462] SavedModel load for tags { serve }; Status: success: OK. Took 90106 

In [38]:
interpreter = tf.lite.Interpreter(model_content=quantized_tflite_model)
interpreter.allocate_tensors()
input_type = interpreter.get_input_details()[0]['dtype']
print('input: ', input_type)
output_type = interpreter.get_output_details()[0]['dtype']
print('output: ', output_type)

input:  <class 'numpy.float32'>
output:  <class 'numpy.float32'>


In [39]:
print("Validation dataset:")
(
    y_pred_val, 
    non_overlap_patritions_f1_scores_val, 
    bootstrap_patritions_f1_scores_val,
    ) = predict_and_print_full_results(quantized_tflite_model, x_val_np, y_val_np, model_format="tf_lite")

print("\nTest dataset:")
(
    y_pred_test, 
    non_overlap_patritions_f1_scores_test, 
    bootstrap_patritions_f1_scores_test,
    ) = predict_and_print_full_results(quantized_tflite_model, x_test_np, y_test_np, model_format="tf_lite")

Validation dataset:
Basic assessment of the whole dataset (without any partitions):
Accuracy: 98.26%
Recall: 96.82%
Precision: 98.07%
F1-score: 97.44%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.9750240315264099

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.9741282992097848

Test dataset:
Basic assessment of the whole dataset (without any partitions):
Accuracy: 98.35%
Recall: 96.51%
Precision: 98.44%
F1-score: 97.46%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.9740913067264898

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.9743646852213748


In [21]:
print("For Test dataset:")
evaluate_time_of_prediction(quantized_tflite_model, x_test_np, y_test_np, model_format="tf_lite")

For Test dataset:
Accuracy: 95.98%
Recall: 94.76%
Precision: 93.13%
F1-score: 93.94%

Time to make a prediction for a single data point
Mean: 0.001 seconds
Max: 0.005 seconds
Min: 0.001 seconds


In [23]:
# Save model and print its size
QAT_MODEL_FILE_NAME = "../spectrogram_models_from_notebooks/cnn/quantized/cnn_mel_speq_qat.tflite"
open(QAT_MODEL_FILE_NAME, "wb").write(quantized_tflite_model)
print("Model file name: ", QAT_MODEL_FILE_NAME)
convert_bytes(get_file_size(QAT_MODEL_FILE_NAME), "KB")

Model file name:  ../spectrogram_models_from_notebooks/cnn/quantized/cnn_mel_speq_qat.tflite
File size: 30.359 Kilobytes
